In [1]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [6]:
def calc_logloss(y, y_pred):
    # изменение
    for i, value in enumerate(y_pred):
        if value == 0:
            y_pred[i] = 0.01
        
        elif value == 1:
            y_pred[i] = 0.99
            
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [7]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.1, 0.1])
calc_logloss(y1, y_pred1)

1.203972804325936

In [8]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05770542575566387

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [9]:
def eval_model(X, y, iterations, eta):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [39]:
W = eval_model(X_st, y, iterations=10000, eta=1e-1)

0 [ 0.45877546 -0.2833519   0.6449505   1.46641523] 1.1785958344356262
1000 [-2.52102182 -0.94535892  0.39937552  3.13420485] 0.37533367618380087
2000 [-4.27828712 -0.98168834 -0.27476935  4.26319069] 0.3266312200993096
3000 [-5.56189789 -1.0318158  -0.73874052  5.15242458] 0.29991853172016747
4000 [-6.58645538 -1.0857742  -1.0912958   5.89903883] 0.28252262722907645
5000 [-7.45579588 -1.13940477 -1.38052844  6.55325919] 0.26979795435357545
6000 [-8.22400342 -1.19158043 -1.63004704  7.14381634] 0.2597485783476198
7000 [-8.92162316 -1.24206841 -1.8526335   7.6879962 ] 0.2513938116008701
8000 [-9.56712984 -1.29090081 -2.05579238  8.19673599] 0.24419878346796198
9000 [-10.17234391  -1.33818162  -2.24422182   8.677278  ] 0.23784703122543333


In [11]:
W

array([-10.744631  ,  -1.38398009,  -2.42084781,   9.13416686])

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [12]:
def calc_pred_proba(x, w):
    return np.squeeze(sigmoid(np.dot(x, w)))

In [13]:
y_pred_proba = calc_pred_proba(X_st, W)
y_pred_proba

array([0.34909794, 0.16499718, 0.99721038, 0.00128184, 0.73112245,
       0.32370941, 0.99918939, 0.0911856 , 0.40772414, 0.99034837])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [14]:
def calc_pred (x, w):
    y_pred_proba = np.squeeze(sigmoid(np.dot(x, w)))
    y_pred = y_pred_proba.copy()
    for i, value in enumerate(y_pred_proba):
        if value >= 0.5:
            y_pred[i] = 1
        else:
            y_pred[i] = 0
    return y_pred

In [30]:
y_pred = calc_pred(X_st, W)
y_pred, y

(array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.]),
 array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.]))

5. Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [24]:
def accuracy(y, predection):
    return sum(y == y_pred) / len(y)

In [25]:
accuracy(y , y_pred)

0.9

In [28]:
def err_matrix(y, predection):
    matrix = np.zeros((2, 2))
    for i, value in enumerate(y):
        if value == 1:
            if predection[i] == value:
                matrix[0][0] += 1
            else:
                 matrix[0][1] += 1
        elif value == 0:
            if predection[i] == value:
                matrix[1][1] += 1
            else:
                 matrix[1][0] += 1
    return matrix

In [32]:
err_matrix = err_matrix(y , y_pred)
err_matrix

array([[4., 1.],
       [0., 5.]])

In [31]:
def precision(err_matrix):
    return err_matrix[0][0] / (err_matrix[0][0] + err_matrix[0][1])

In [33]:
precision(err_matrix)

0.8

In [34]:
def recall(err_matrix):
    return err_matrix[0][0] / (err_matrix[0][0] + err_matrix[1][0])

In [35]:
recall(err_matrix)

1.0

In [36]:
def F1(err_matrix):
    return 2 * precision(err_matrix) * recall(err_matrix) / (precision(err_matrix) + recall(err_matrix))

In [37]:
F1(err_matrix)

0.888888888888889

6. Могла ли модель переобучиться? Почему?

кажется нет не могла